## 1,レース情報をスクレイピング

jupyter notebook で [ImportError: lxml not found, please install it]エラーが出た場合は [pip3 install lxml] でインストール後プロジェクトを再起動


In [189]:
import pandas as pd
from tqdm import tqdm
import time

url >> race_id=2022 09 01 11 01 : 年  競馬場(01:札幌,02:函館,03:福島,04:新潟,05:東京,06:中山,07:中京,08:京都,09:阪神,10:小倉) 何回(今年その競馬場でのレース回数) 何日目(何レース目)

In [190]:
pd.set_option('display.max_rows',5)
pd.options.display.width = 200
pd.options.display.column_space = 100

スクレイピング

In [191]:
df = pd.read_csv("test.csv")
already = df.index.unique()
already = list(map(str, already))
already

['202101010101',
 '202101010102',
 '202101010103',
 '202101010104',
 '202101010105',
 '202101010106',
 '202101010107',
 '202101010108',
 '202101010109',
 '202101010110',
 '202101010111',
 '202101010112']

In [192]:
def scrape_race_results(race_id_lists,par_race_results={}):
  race_result_dict = par_race_results
  for _race_id in tqdm(race_id_lists):
    if _race_id in already:
      continue
    time.sleep(2)
    try:
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      race_result_dict[_race_id] = pd.read_html(url)[0]
      # print(race_result_dict[race_id])
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_result_dict

# race_result = {}

dictのkeyをrace_idに変換


In [193]:
def get_house_data():
  race_id_lists = []

  for 年 in range(2021,2022):
    for 競馬場 in range(1,3): # 1,11
      for 回 in range(1,2): # 1,11
        for 回目 in range(1,2): # 1,11
          for レース回 in range(1,13): # 1,13
            race_id = f"{年}{str(競馬場).zfill(2)}{str(回).zfill(2)}{str(回目).zfill(2)}{str(レース回).zfill(2)}"
            race_id_lists.append(race_id)
            
  race_result = scrape_race_results(race_id_lists)
  for key in race_result.keys():
    race_result[key].index = [key] * len(race_result[key])
    
  results = pd.concat([race_result[key] for key in race_result.keys()],sort=False)
  return results

新規でdata取得するとき

In [194]:
house_data = get_house_data()

100%|██████████| 24/24 [00:27<00:00,  1.15s/it]


追加でdata取得するとき

In [195]:
len(house_data)

149

In [196]:
house_data

,着順,枠,馬番,馬名,性齢,斤量,騎手,タイム,着差,人気,単勝オッズ,後3F,コーナー通過順,厩舎,馬体重(増減),Unnamed: 15,Unnamed: 16,Unnamed: 17
202102010101,1,4,4,ナムラリコリス,牝2,52.0,△泉谷,1:09.3,NaN,1,1.4,35.7,2-2,栗東大橋,466(+2),NaN,NaN,NaN
202102010101,2,1,1,プラソン,牝2,54.0,藤岡佑,1:09.4,1/2,2,5.5,35.9,1-1,栗東寺島,430(+2),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202102010112,10,7,9,ウエストンバート,牡3,54.0,横山武,1:48.1,2.1/2,4,6.8,36.3,2-2-2-2,栗東吉岡,522(-6),NaN,NaN,NaN
202102010112,11,8,10,ザイラ,牝4,55.0,吉田隼,1:48.3,1.1/4,10,45.9,35.6,9-9-8-10,栗東奥村豊,484(0),NaN,NaN,NaN


保存する

In [200]:

# house_data.to_csv('test1.csv')
# house_data.to_csv('test2.csv')

In [219]:
# pd.set_option('display.max_rows',None)
# df = pd.read_csv("test.csv")
# df1 = df.iloc[:,:15]
# print(df1.columns)
# df1["厩舎"]

Index(['Unnamed: 0', '着順', '枠', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '人気', '単勝オッズ', '後3F', 'コーナー通過順', '厩舎'], dtype='object')


0       栗東大橋
1       栗東寺島
2       美浦深山
3      美浦伊藤大
4       栗東加用
5       美浦手塚
6       美浦田村
7      栗東飯田祐
8      美浦高橋裕
9       栗東梅田
10     栗東清水久
11      美浦手塚
12       美浦堀
13     栗東大根田
14     美浦田中博
15      美浦小島
16       栗東宮
17      栗東野中
18      栗東平田
19     栗東高柳大
20      栗東岡田
21      栗東牧浦
22     栗東高橋忠
23      栗東浜田
24     栗東池添兼
25      美浦新開
26     栗東中内田
27      栗東大橋
28      美浦竹内
29      美浦栗田
30     美浦大江原
31      美浦稲垣
32      美浦高木
33      栗東友道
34     美浦加藤征
35     栗東藤原英
36      栗東武英
37      栗東坂口
38      美浦岩戸
39      美浦深山
40     美浦清水英
41     栗東藤沢則
42      美浦田村
43     美浦伊藤伸
44      栗東吉田
45      美浦本間
46     栗東杉山佳
47     美浦伊藤圭
48      美浦杉浦
49      美浦小野
50      美浦新開
51      栗東浅見
52      美浦岩戸
53      栗東今野
54     美浦蛯名利
55      栗東加用
56     栗東池添兼
57      美浦堀井
58      栗東須貝
59      栗東松下
60     栗東安田隆
61      栗東音無
62      美浦国枝
63      栗東本田
64      栗東笹田
65      美浦堀井
66     美浦田中清
67     美浦古賀慎
68      美浦杉浦
69       栗東宮
70      栗東渡辺
71      栗東吉村
72     美浦田中博
73      栗東森田
74     栗東高柳大
75     美浦高橋祥
76      栗東新谷